<a href="https://colab.research.google.com/github/Ganeshram5628/Fake-News-Detection/blob/main/SVM_and_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import spacy
import nltk
from nltk.stem import WordNetLemmatizer
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train_news.csv')

In [ ]:
type(df)
df.head()

,Unnamed: 0,id,headline,written_by,news,label
0,0,9653,Ethics Questions Dogged Agriculture Nominee as...,Eric Lipton and Steve Eder,"WASHINGTON — In Sonny Perdue’s telling, Geo...",0
1,1,10041,U.S. Must Dig Deep to Stop Argentina’s Lionel ...,David Waldstein,HOUSTON — Venezuela had a plan. It was a ta...,0
2,2,19113,Cotton to House: ’Do Not Walk the Plank and Vo...,Pam Key,"Sunday on ABC’s “This Week,” while discussing ...",0
3,3,6868,"Paul LePage, Besieged Maine Governor, Sends Co...",Jess Bidgood,"AUGUSTA, Me. — The beleaguered Republican g...",0
4,4,7596,A Digital 9/11 If Trump Wins,Finian Cunningham,Finian Cunningham has written extensively on...,1


In [ ]:
df.shape

(20800, 6)

In [ ]:
df.values

array([[0, 9653,
        'Ethics Questions Dogged Agriculture Nominee as Georgia Governor - The New York Times',
        'Eric Lipton and Steve Eder',
        'WASHINGTON  —   In Sonny Perdue’s telling, Georgians were growing weary of the corruption and scandals in their state when he took over as governor in 2003  —   a time when he gave his own version of a “drain the swamp” pledge. “My first goal as governor is to restore public trust in state government by changing the culture of state government,” Mr. Perdue told a leadership conference at Kennesaw State University in April 2003. “Our form of government depends on a mutual bond of trust between the people and their government. But people have become cynical about their government. ” But Mr. Perdue, a Republican who is awaiting confirmation to serve as President Trump’s agriculture secretary, became a target of frequent criticism that he was failing to honor his own ethics pledge during his eight years as governor. The criticism ce

In [ ]:
df.describe

<bound method NDFrame.describe of        Unnamed: 0     id                                           headline  \
0               0   9653  Ethics Questions Dogged Agriculture Nominee as...   
1               1  10041  U.S. Must Dig Deep to Stop Argentina’s Lionel ...   
2               2  19113  Cotton to House: ’Do Not Walk the Plank and Vo...   
3               3   6868  Paul LePage, Besieged Maine Governor, Sends Co...   
4               4   7596                       A Digital 9/11 If Trump Wins   
...           ...    ...                                                ...   
20795       20795   5671                                                NaN   
20796       20796  14831        Albert Pike and the European Migrant Crisis   
20797       20797  18142  Dakota Access Caught Infiltrating Protests to ...   
20798       20798  12139  How to Stretch the Summer Solstice - The New Y...   
20799       20799  15660  Emory University to Pay for ’100 Percent’ of U...   

                       written_by  \
0      Eric Lipton and Steve Eder   
1                 David Waldstein   
2                         Pam Key   
3                    Jess Bidgood   
4               Finian Cunningham   
...                           ...   
20795              NeverSurrender   
20796               Rixon Stewart   
20797                 Eddy Lavine   
20798              Alison S. Cohn   
20799                Tom Ciccotta   

                                                    news  label  
0      WASHINGTON  —   In Sonny Perdue’s telling, Geo...      0  
1      HOUSTON  —   Venezuela had a plan. It was a ta...      0  
2      Sunday on ABC’s “This Week,” while discussing ...      0  
3      AUGUSTA, Me.  —   The beleaguered Republican g...      0  
4        Finian Cunningham has written extensively on...      1  
...                                                  ...    ...  
20795  No, you'll be a dog licking of the vomit of yo...      1  
20796  By Rixon Stewart on November 5, 2016 Rixon Ste...      1  
20797  posted by Eddie You know the Dakota Access Pip...      1  
20798  It’s officially summer, and the Society Boutiq...      0  
20799  Emory University in Atlanta, Georgia, has anno...      0  

[20800 rows x 6 columns]>

In [ ]:
df.isnull().sum()

Unnamed: 0       0
id               0
headline       558
written_by    1957
news            39
label            0
dtype: int64

In [ ]:
df = df.fillna('')

In [ ]:
df.isnull().sum()

Unnamed: 0    0
id            0
headline      0
written_by    0
news          0
label         0
dtype: int64

In [ ]:
df['content'] = df['written_by'] + ' ' + df['headline']

In [ ]:
x = df.drop('label',axis=1)
y = df['label']

In [ ]:
x

,Unnamed: 0,id,headline,written_by,news,content
0,0,9653,Ethics Questions Dogged Agriculture Nominee as...,Eric Lipton and Steve Eder,"WASHINGTON — In Sonny Perdue’s telling, Geo...",Eric Lipton and Steve Eder Ethics Questions Do...
1,1,10041,U.S. Must Dig Deep to Stop Argentina’s Lionel ...,David Waldstein,HOUSTON — Venezuela had a plan. It was a ta...,David Waldstein U.S. Must Dig Deep to Stop Arg...
2,2,19113,Cotton to House: ’Do Not Walk the Plank and Vo...,Pam Key,"Sunday on ABC’s “This Week,” while discussing ...",Pam Key Cotton to House: ’Do Not Walk the Plan...
3,3,6868,"Paul LePage, Besieged Maine Governor, Sends Co...",Jess Bidgood,"AUGUSTA, Me. — The beleaguered Republican g...","Jess Bidgood Paul LePage, Besieged Maine Gover..."
4,4,7596,A Digital 9/11 If Trump Wins,Finian Cunningham,Finian Cunningham has written extensively on...,Finian Cunningham A Digital 9/11 If Trump Wins
...,...,...,...,...,...,...
20795,20795,5671,,NeverSurrender,"No, you'll be a dog licking of the vomit of yo...",NeverSurrender
20796,20796,14831,Albert Pike and the European Migrant Crisis,Rixon Stewart,"By Rixon Stewart on November 5, 2016 Rixon Ste...",Rixon Stewart Albert Pike and the European Mig...
20797,20797,18142,Dakota Access Caught Infiltrating Protests to ...,Eddy Lavine,posted by Eddie You know the Dakota Access Pip...,Eddy Lavine Dakota Access Caught Infiltrating ...
20798,20798,12139,How to Stretch the Summer Solstice - The New Y...,Alison S. Cohn,"It’s officially summer, and the Society Boutiq...",Alison S. Cohn How to Stretch the Summer Solst...


In [ ]:
y

0        0
1        0
2        0
3        0
4        1
        ..
20795    1
20796    1
20797    1
20798    0
20799    0
Name: label, Length: 20800, dtype: int64

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
wnl = WordNetLemmatizer()
def lemmatize(content):
    lemmatized_content = re.sub('[^a-zA-Z]',' ',content)
    lemmatized_content = lemmatized_content.lower()
    lemmatized_content = lemmatized_content.split()
    lemmatized_content = [wnl.lemmatize(word) for word in lemmatized_content if not word in stopwords.words('english')]
    lemmatized_content = ' '.join(lemmatized_content)
    return lemmatized_content

In [ ]:
df['content'] = df['content'].apply(lemmatize)

In [ ]:
x = df['content'].values
y= df['label'].values

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,stratify =y,random_state=100)

In [ ]:
vec = TfidfVectorizer()
vec.fit(x_train)
x_train = vec.transform(x_train)
print(x_train)

  (0, 17765)	0.08530496398002532
  (0, 16070)	0.08366431084777429
  (0, 15503)	0.3022248194774148
  (0, 15113)	0.26776220048835897
  (0, 14933)	0.3022248194774148
  (0, 14853)	0.2935936804353523
  (0, 13488)	0.2849625413932897
  (0, 10778)	0.24298768078540697
  (0, 10738)	0.08248411625915822
  (0, 7210)	0.34721521380964476
  (0, 5954)	0.31025626030306974
  (0, 5878)	0.23808495521369846
  (0, 4865)	0.366973826411289
  (0, 3573)	0.29987987364685703
  (1, 16834)	0.3081383859065355
  (1, 16555)	0.29177889630060116
  (1, 14422)	0.402749748725721
  (1, 10651)	0.3391377925682055
  (1, 10377)	0.30744872012297747
  (1, 7527)	0.30543654514992685
  (1, 3699)	0.3769502728312532
  (1, 19)	0.46128765597175364
  (2, 17765)	0.08328444695064645
  (2, 17501)	0.23933083870529073
  (2, 16376)	0.2797545586682315
  :	:
  (14557, 2027)	0.2956598750233664
  (14557, 279)	0.35554515731019026
  (14558, 17765)	0.10116502066579444
  (14558, 16070)	0.09921933426859438
  (14558, 15236)	0.4034068049637849
  (14558, 1

In [ ]:
print(x_test,y_test)

['nicholas lemann edward snowden spy new book call one new york time'
 'admin leaked memo expose shady dealing clinton foundation donor bill profit activity'
 'kirk johnson idaho town buck perception rural struggle new york time'
 ...
 'erik eckholm south carolina law disrupting school face legal challenge new york time'
 'michael steinberger electric infuriating nick kyrgios new york time'
 'deplorable revbo kim dotcom know hillary clinton deleted email get legally'] [0 1 0 ... 0 0 1]


In [ ]:
x_test = vec.transform(x_test)
print(x_test)

  (0, 17765)	0.11848605111247087
  (0, 16070)	0.11620723283723837
  (0, 15088)	0.3897158848051009
  (0, 14809)	0.45069271958109297
  (0, 11144)	0.29607480071268016
  (0, 10778)	0.33750264254237294
  (0, 10738)	0.22913595549581545
  (0, 4906)	0.3897158848051009
  (0, 2246)	0.28798526886494114
  (0, 1818)	0.3585651167940168
  (1, 14320)	0.3982838300456382
  (1, 12400)	0.30982501909975696
  (1, 9950)	0.29659966223138357
  (1, 8986)	0.2678967736338486
  (1, 6108)	0.26135386882208855
  (1, 5513)	0.27767762639937466
  (1, 4587)	0.289259516717743
  (1, 3832)	0.3377306669954115
  (1, 2885)	0.15323429783493367
  (1, 1591)	0.21452618509442514
  (1, 187)	0.2058794732380627
  (1, 139)	0.36139457008633813
  (2, 17765)	0.09233284090373949
  (2, 16204)	0.2898218755888103
  (2, 16070)	0.0905570220349357
  :	:
  (6237, 10738)	0.09109911003676326
  (6237, 9025)	0.29914782520253563
  (6237, 8954)	0.2495326722177757
  (6237, 5564)	0.2563663044131276
  (6237, 5265)	0.34266075068490354
  (6237, 4864)	0.3691

In [ ]:
model = SVC(kernel = 'rbf', gamma=1, C=100)
model.fit(x_train,y_train)

SVC(C=100, gamma=1)

In [ ]:
predict = model.predict(x_test)
acc = accuracy_score(y_test,predict)

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, predict))
print("Precision:",metrics.precision_score(y_test, predict))
print("Recall:",metrics.recall_score(y_test, predict))
print("F1 Score:",metrics.f1_score(y_test, predict))

Accuracy: 0.9908653846153846
Precision: 0.9879096404708877
Recall: 0.9939180537772087
F1 Score: 0.9909047391096218


In [ ]:
print(metrics.classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3116
           1       0.99      0.99      0.99      3124

    accuracy                           0.99      6240
   macro avg       0.99      0.99      0.99      6240
weighted avg       0.99      0.99      0.99      6240



In [ ]:
ip = input("enter a string:")
l = [lemmatize(ip)]
print(l)

enter a string:Putin: No difference between Clinton, Trump
['putin difference clinton trump']


In [ ]:
l =vec.transform(l)
type(l)

scipy.sparse._csr.csr_matrix

In [ ]:
prediction = model.predict(l)

In [ ]:
if(prediction[0] == 0):
  print("the news is real")
else:
  print('fake')

fake


different dataset

In [ ]:
fd = pd.read_csv("/content/drive/MyDrive/train 2.csv")

In [ ]:
fd.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
fd.tail()

,id,title,author,text,label
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20799,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [ ]:
fd.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
fd = fd.dropna()

In [ ]:
fd.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
l = fd.drop('label',axis=1)
m = fd['label']

In [ ]:
l

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
m

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [ ]:
fd['title'] = fd['title'].apply(lemmatize)

In [ ]:
l = fd['title'].values
m = fd['label'].values

In [ ]:
l_train,l_test,m_train,m_test = train_test_split(l,m,test_size=0.3,stratify =m,random_state=100)

In [ ]:
vec1 = TfidfVectorizer()
vec1.fit(l_train)
l_train = vec1.transform(l_train)
print(l_train)

  (0, 14676)	0.08846242456928302
  (0, 14056)	0.40000928297255284
  (0, 13264)	0.08739560910867572
  (0, 12529)	0.41733986792515027
  (0, 11330)	0.28929483612375695
  (0, 11037)	0.37038245231036193
  (0, 8800)	0.08573857195587382
  (0, 8047)	0.3877130372629594
  (0, 6874)	0.27415665386649796
  (0, 6091)	0.1634422070177021
  (0, 2351)	0.164065368882015
  (0, 164)	0.3781753281103298
  (1, 14683)	0.6580341065920556
  (1, 9510)	0.7529881237852263
  (2, 14676)	0.12810945414001393
  (2, 13542)	0.164848910259778
  (2, 13264)	0.1265645140483088
  (2, 8800)	0.24832965421174474
  (2, 7587)	0.3489183916454579
  (2, 4961)	0.3297534683986929
  (2, 3933)	0.5112824143192302
  (2, 959)	0.5363802091062762
  (2, 435)	0.3148519990511224
  (3, 14676)	0.11690401755488174
  (3, 13264)	0.11549420978687308
  :	:
  (12796, 2351)	0.19680114696336742
  (12796, 2075)	0.500611221252149
  (12797, 14676)	0.12240846139201413
  (12797, 13264)	0.12093227260612033
  (12797, 13056)	0.3725090905467189
  (12797, 11054)	0.3

In [ ]:
print(l_test,m_test)

['happened hillary supporting mit professor decided analyze email'
 'anonymous issue warning nd governor dapl watch'
 'diez consejos para preparar tu casa de cara la llegada de tu primer hijo'
 ... 'saint fan might created mardi gras float mock falcon breitbart'
 'mosul ceta trump tuesday briefing new york time'
 'review knife kitchen city describes syrian hell new york time'] [1 1 1 ... 0 0 0]


In [ ]:
l_test = vec1.transform(l_test)
print(l_test)

  (0, 12825)	0.4118467704985972
  (0, 10198)	0.39784776625684876
  (0, 8383)	0.4371823589941155
  (0, 6091)	0.19954426701184821
  (0, 5853)	0.38207363547893636
  (0, 4223)	0.26666349575859033
  (0, 3251)	0.47335333536701013
  (1, 14275)	0.29508633038093457
  (1, 14257)	0.3733622815406917
  (1, 8720)	0.4156540826129666
  (1, 6832)	0.3644017232758522
  (1, 5600)	0.38029302645939206
  (1, 3159)	0.4089404985823147
  (1, 531)	0.395110889082305
  (2, 13570)	0.5861666148378036
  (2, 10138)	0.3004749671488478
  (2, 9420)	0.24886122003935077
  (2, 7297)	0.20711515107566206
  (2, 3605)	0.3234355594281024
  (2, 3197)	0.4076468494042099
  (2, 1990)	0.2930833074189018
  (2, 1941)	0.3234355594281024
  (3, 11628)	0.34558387735546114
  (3, 11260)	0.41072690542871126
  (3, 11113)	0.4181657107634137
  :	:
  (5483, 8410)	0.34488341227112357
  (5483, 8280)	0.33228502003317484
  (5483, 5023)	0.40616843044631273
  (5483, 4749)	0.31683880980111073
  (5483, 4728)	0.44823198796399644
  (5483, 2922)	0.369973235

In [ ]:
model2 = SVC(kernel = 'rbf', gamma=1, C=100)
model2.fit(l_train,m_train)

SVC(C=100, gamma=1)

In [ ]:
predict2 = model2.predict(l_test)
acc = accuracy_score(m_test,predict2)

In [ ]:
print("Accuracy:",metrics.accuracy_score(m_test, predict2))
print("Precision:",metrics.precision_score(m_test, predict2))
print("Recall:",metrics.recall_score(m_test, predict2))
print("F1 Score:",metrics.f1_score(m_test, predict2))

Accuracy: 0.9298213634706526
Precision: 0.8764172335600907
Recall: 0.9755994951619689
F1 Score: 0.9233525781405535


In [ ]:
print(metrics.classification_report(m_test, predict2))

              precision    recall  f1-score   support

           0       0.98      0.89      0.94      3109
           1       0.88      0.98      0.92      2377

    accuracy                           0.93      5486
   macro avg       0.93      0.94      0.93      5486
weighted avg       0.93      0.93      0.93      5486



In [ ]:
ip = input("enter a string:")
n = [lemmatize(ip)]
print(n)

enter a string:Putin: No difference between Clinton, Trump
['putin difference clinton trump']


In [ ]:
n =vec1.transform(n)
type(n)

scipy.sparse._csr.csr_matrix

In [ ]:
prediction2 = model2.predict(n)

In [ ]:
if(prediction2[0] == 0):
  print("the news is real")
else:
  print('fake')

fake
